In [ ]:
import os
import numpy as np
import pickle
import scipy.io as sio
import scipy.stats
%matplotlib inline
import matplotlib.pyplot as plt

import yaml
with open('./util/config__drama_data.yaml', 'r') as yml:
    config = yaml.safe_load(yml)
from util import util_pycortex as utlpy
from util import util_dataload as udl
from util import util_visualization as uvis


# (Demo 1) Pycortex visualization
## Param: Setting parameters
subject_name: S01–S06 <br>
stat_path: <br>
- File path for statistics (or estimates).
- Visualized data should be stored as a dictionary with the key 'stat'. The data array should have a size of (tvoxels,).
- "tvoxels" denotes voxel indices of the cortex, and can be obtained by the following command.
> from util import util_dataload as udl<br>
> load_items = udl.set_load_items(config, subjectID=loadInfoSbID, movID=0) # loadInfoSbID: 0)S01, 1)S02, ..., 5)S06<br>
> tvoxels = udl.load_mask_data(load_items['mask_path']) # The obtained voxel indices follow Python order.<br>

In [ ]:
subject_name = 'S01'
stat_path = './demo_files/ridge_stats__DM01.pkl'
saveFigName = 'visualization_test' # Save as .png file

with_rois = True
with_sulci = True
with_curvature = True

## Main: Visualization

In [ ]:
###
### Custom

dataInfo = utlpy.get_dataInfo(config, subject_name)

# Other params
max_stat = 0.3 # Setting max value to show result
with_colorbar = True
Sigs = [] # [] is ok. [data array with size:(sigTvoxelIDs, )] IDs of tvoxels showing significance
integrationScale = 1 # Arrange stat scales when integrating colors

# Load stats
with open(stat_path, 'rb') as f:
    dt = pickle.load(f)
Stats = [dt['stats']]

if len(Sigs)== 0: get_dt3d_sig = False
else: get_dt3d_sig = True

# Reshape 1d stats to 3d. 
dt3d_stats_show, dt3d_sig_show, _ = utlpy.get_dt3d_vol(dataInfo, Stats, Sigs, nLaps=1, normData=False, integrationScale=integrationScale, get_dt3d_sig=get_dt3d_sig) # 

# Set mask if "get_dt3d_sig == True."
if get_dt3d_sig == True:
    sig_mask = np.sum(dt3d_sig_show, axis=0)
    sig_mask[sig_mask>0] = 1
    mask = sig_mask #None

# Show the flattened cortical map
uvis.pycortex_visualization(dataInfo, dt3d_stats_show, dt3d_sig_show, \
                           showType = 'stats', colorType = 'single', max_stat = max_stat, \
                           mask = [], saveFigName = [], \
                           with_rois=with_rois, with_sulci=with_sulci, with_curvature=with_curvature, with_colorbar=with_colorbar)


# (Demo 2) Visualization of encoding and froi results in our analyses.

## Param: Setting parameters
subject_name: S01–S06<br>
result: 'banded__obj_speech_story' or 'loc_{no}' {no}:1–9 (e.g. loc_3)

In [ ]:
subject_name = 'S01' # S01, S02, ..., S06
resultType = 'banded__obj_speech_story' # banded__obj_speech_story, loc_{no}:{no}=[1,2,...,11] (e.g. loc_3)

with_rois = True
with_sulci = True
with_curvature = True

## Main: Visualization

In [ ]:

dataInfo = utlpy.get_dataInfo(config, subject_name)

###
### Banded ridge

if resultType == 'banded__obj_speech_story':
    
    dt = uvis.get_results_visualization_demo(config, dataInfo['subjectName'], resultType)
    dt3d_stats_show, dt3d_sig_show, mask, showType = uvis.get_bandedridge_dt3d(dataInfo, dt)
    
    saveFigName = 'visualization_test'
    with_colorbar = False
    colorType = 'rgb'
    max_stat = 0.02
    
    uvis.pycortex_visualization(dataInfo, dt3d_stats_show, dt3d_sig_show, \
                               showType = 'stats', colorType = colorType, max_stat = max_stat, \
                               mask = mask, saveFigName = saveFigName, \
                               with_rois=with_rois, with_sulci=with_sulci, with_curvature=with_curvature, with_colorbar=with_colorbar)

###
### functional ROIs

if 'loc_' in resultType:
    
    dt, locName = uvis.get_results_visualization_demo(config, dataInfo['subjectName'], resultType)
    dt3d_stats_show, dt3d_sig_show, mask, showType, colorType = uvis.get_froi_dt3d(dataInfo, dt, locName)
    
    saveFigName = 'visualization_test'
    with_colorbar = False
    max_stat = uvis.set_max_stat(locName, colorType)
    
    uvis.pycortex_visualization(dataInfo, dt3d_stats_show, dt3d_sig_show, \
                               showType = 'stats', colorType = colorType, max_stat = max_stat, \
                               mask = mask, saveFigName = saveFigName, \
                               with_rois=with_rois, with_sulci=with_sulci, with_curvature=with_curvature, with_colorbar=with_colorbar)



## (Demo3) Getting voxel indices in a specific ROI.
Available roi names: 'V1', 'V2', 'V3', 'FFA', 'OFA', 'PPA', 'MT+', 'AC', 'S1H', 'S1M', 'S1F', 'M1H', 'M1M', 'M1F', 'IPS', 'FEF'<br>
Refer to the following Pycortex page for available options: https://gallantlab.org/pycortex/generated/cortex.utils.get_roi_masks.html

In [ ]:
roi = "MT+"
subject_name = 'S01' # S01, S02, ..., S06

In [ ]:
import cortex
dataInfo = utlpy.get_dataInfo(config, subject_name)

roi_masks = cortex.utils.get_roi_masks(dataInfo['subjectNamePycortex'], dataInfo['dataSetName'],
                                       roi_list=[roi], gm_sampler='cortical-conservative')

roi_data = cortex.Volume(roi_masks[roi], dataInfo['subjectNamePycortex'], dataInfo['dataSetName'], vmin=0, vmax=1, cmap="inferno")
cortex.quickflat.make_figure(roi_data, thick=1, with_curvature=True, with_colorbar=True)

plt.show()

### Getting voxel IDs in the roi.
The "roi_mask_voxelIDs" in the code below corresponds to the voxel IDs in the roi.

In [ ]:

roi_mask_3d = roi_masks[roi]
roi_mask_voxelIDs = np.where(roi_mask_3d.flatten()>0)[0]


### Showing voxel IDs of the roi in the epi space.

In [ ]:

datasize = dataInfo['datasize']
voxels_in_epi_space__flatten = np.zeros(np.prod(datasize))
voxels_in_epi_space__flatten[roi_mask_voxelIDs] = 0.8
voxels_in_epi_space__3d = voxels_in_epi_space__flatten.reshape([datasize[2], datasize[1], datasize[0]])

# cortex 3d
subjectID = np.where(np.array(config['subjectInfo']['names'])==subject_name)[0][0]
dtinfo = udl.set_load_items(config, subjectID=subjectID, movID=0)
tvoxels = udl.load_mask_data(dtinfo['mask_path'])
cortex__flatten = np.zeros(np.prod(datasize))
cortex__flatten[tvoxels] = 0.2
cortex__3d = cortex__flatten.reshape([datasize[2], datasize[1], datasize[0]])

# Show the voxel IDs in the slice image.
vol_value = cortex__3d + voxels_in_epi_space__3d
plt.imshow(vol_value[25,:,:])
plt.show()
